In [1]:
import os
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def load_price_timeseries(file: str) -> pd.Series:
    df = pd.read_csv(file)
    df.index = pd.to_datetime(df["Date"], format="%d/%m/%Y %H:%M")
    return df["Intraday Continuous 15 minutes ID1-Price"]  # * 1e-6 # €/MWh -> €/Wh

In [3]:
def load_results(dir):
    res = {}
    files = [f for f in os.listdir(dir) if f.endswith(".parquet")]
    for file in files:
        name, _ = os.path.splitext(file)
        res[name] = pd.read_parquet(dir + file)

    return res

In [4]:
def exctract_value(string, key):
    match = re.search(fr"{key}=([\d.]+)", string)
    if match:
        value = match.group(1)
    return float(value)

In [5]:
def calc_fec(df):
    power = df["power_sim"]
    power_pos = power[power > 0].sum() * (1 / 60)
    power_neg = power[power < 0].abs().sum() * (1 / 60)
    return (power_pos + power_neg) / 2 / 180e3

In [6]:
def calc_roundtrip_efficiency(res):
    p = res["power_sim"]
    e_pos = p[p > 0].abs().sum() * (1 / 60)  # Wh
    e_neg = p[p < 0].abs().sum() * (1 / 60)  # Wh

    delta_soc = res["soc_sim"].iloc[-1] - res["soc_sim"].iloc[0]
    delta_e = delta_soc * 180e3  # Wh

    return abs(e_neg) / (e_pos - delta_e)

In [7]:
def calc_revenue(df, price):
    price = price.resample("1Min").ffill()
    df = df.join(price)
    return -1 * sum(df["power_sim"] * df["Intraday Continuous 15 minutes ID1-Price"]) * (1/60) * 1e-6  # W -> MWh

In [18]:
def analyse_results_lp(res, price):
    out = pd.DataFrame()
    for (id, df) in res.items():
        if "LP" in id:
            data = dict(
                r = exctract_value(id, "r"),
                eff = exctract_value(id, "eff"),
                rev = calc_revenue(df, price),
                throughput = calc_fec(df),
                rountrip_eff = calc_roundtrip_efficiency(df),
            )
            out = pd.concat([out, pd.DataFrame(data=[data])])

    return out

In [19]:
def analyse_results_nl(res, price):
    out = pd.DataFrame()
    for (id, df) in res.items():
        if "NL" in id:
            data = dict(
                r = exctract_value(id, "r"),
                r_opt = exctract_value(id, "r_opt"),
                rev = calc_revenue(df, price),
                throughput = calc_fec(df),
                rountrip_eff = calc_roundtrip_efficiency(df),
            )
            out = pd.concat([out, pd.DataFrame(data=[data])])

    return out

In [33]:
def plot_trading_power(res, price):
    fig, ax = plt.subplots(2, 2, height_ratios=[1, 3], width_ratios=[3, 1])

    df = res.join(price)
    power = df["power_sim"] / 100e3
    price = df["Intraday Continuous 15 minutes ID1-Price"]

    idx = power > 0
    # ax.scatter(price.loc[idx], power.loc[idx])
    ax[1, 0].scatter(power.loc[idx], price.loc[idx], alpha=0.5)
    ax[0, 0].hist(power.loc[idx], alpha=0.5, label="Charge")
    ax[1, 1].hist(price.loc[idx], orientation="horizontal", alpha=0.5)

    idx = power < 0
    # ax.scatter(price.loc[idx], power.loc[idx])
    ax[1, 0].scatter(power.loc[idx], price.loc[idx], alpha=0.5)
    ax[0, 0].hist(power.loc[idx], alpha=0.5, label="Discharge")
    ax[1, 1].hist(price.loc[idx], orientation="horizontal", alpha=0.5)

    ax[0, 1].set_visible(False)
    ax[0, 0].xaxis.set_visible(False)
    ax[1, 1].yaxis.set_visible(False)

    # ax[0,0].set_ylabel("Hist")
    # ax[1,1].set_xlabel("Hist")
    ax[1, 0].set_xlabel("Power in p.u.")
    ax[1, 0].set_ylabel("Price in €/MWh")

    fig.tight_layout(h_pad=0.1, w_pad=0.5)

    fig.legend(loc="center", bbox_to_anchor=(0.87, 0.85))

    return fig

In [14]:
price = load_price_timeseries("data/intraday_prices/electricity_prices_germany_2021.csv")

In [17]:
dir = "results/degradation/"
res = load_results(dir)

In [20]:
df_lp = analyse_results_lp(res, price)
df_nl = analyse_results_nl(res, price)

In [29]:
df_lp[df_lp.r == 3.0]

,r,eff,rev,throughput,rountrip_eff
0,3.0,0.85,254.026912,14.059955,0.855119
0,3.0,0.90,284.749437,16.925499,0.854086
0,3.0,0.92,289.824443,17.376979,0.853156
0,3.0,0.95,309.896635,19.604911,0.852537


In [31]:
df_nl[df_nl.r == 3.0]

,r,r_opt,rev,throughput,rountrip_eff
0,3.0,1.0,280.440712,16.599803,0.860276
0,3.0,1.5,278.268461,16.350672,0.860159
0,3.0,2.0,276.689614,16.151995,0.861616
0,3.0,3.0,270.054185,15.510526,0.864488


In [ ]:
plot_trading_power(res["2021 "])